In [1]:
cd ..

/home/admin-hieunn/important/VNUIS-Chatbot


/home/admin-hieunn/important/VNUIS-Chatbot/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from selenium.webdriver.common.by import By
from twocaptcha import TwoCaptcha
from selenium import webdriver
import time
driver = webdriver.Chrome()
url = "https://2captcha.com/demo/normal"
driver.get(url)

from selenium.webdriver.common.by import By
from twocaptcha import TwoCaptcha
from selenium import webdriver
import time

driver = webdriver.Chrome()
url = "https://2captcha.com/demo/normal"
driver.get(url)

imgResults = driver.find_elements(By.XPATH,"//img[contains(@class,'_2hXzbgz7SSP0DXCyvKWcha')]") 
solver = TwoCaptcha(Your_2Captcha_API_key)
result = solver.normal(imgResults[0].get_attribute("src"))
print ("solved: " + str(result))

captchafield = driver.find_element(By.XPATH,"//input[contains(@class,'_26Pq0m_qFk19UXx1w0U5Kv')]")
captchafield.send_keys(result["code"])

button = driver.find_element(By.XPATH,"//button[contains(@class, 'l2z7-tVRGe-3sq5kU4uu5 _2xjDiWmBxfqem8nGQMmGci _2HIb5VBFp6Oi5_JoLdEcl6 _2vbG_IBm-DpI5KeEAHJkRy')]")
button.click()
time.sleep(10)

messagefield=driver.find_element(By.XPATH,"//p[contains(@class,'_2WOJoV7Dg493S8DW_GobSK')]")
print (messagefield.text)

In [ ]:
import io
import json
import logging
import os
import re
from io import BytesIO
from threading import Thread
import base64
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import torch
from dotenv import load_dotenv
from pydub import AudioSegment
import time
# from huggingface_hub import login
# Opitimized
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

# from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from sentence_transformers import SentenceTransformer, util
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoModelForSpeechSeq2Seq,
    AutoProcessor,
    AutoTokenizer,
    TextIteratorStreamer,
    pipeline,
)
from TTS.tts.configs.xtts_config import XttsConfig  # type: ignore
from TTS.tts.models.xtts import Xtts  # type: ignore

from routes.Xu_ly_text import Xu_ly_text_de_doc
from vi_cleaner.vi_cleaner import ViCleaner  # type: ignore

# Configure logging instead of using print statements
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def chrome():
    # Configure Chrome options
    options = Options()
    # options.add_argument("--headless=new")  # Use the new headless mode
    # options.add_argument("--disable-extensions")
    # options.add_argument("--disable-gpu")
    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")
    # options.add_argument("--disable-plugins")
    # options.add_argument("--disable-popup-blocking")
    # options.add_argument("--disable-translate")
    # options.add_argument("--ignore-certificate-errors")
    # options.add_argument("--allow-insecure-localhost")
    options.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )

    # # Disable image loading to speed up page loads
    # prefs = {
    #     "profile.managed_default_content_settings.images": 2,
    #     "profile.default_content_setting_values.stylesheets": 2,
    #     "profile.default_content_setting_values.cookies": 2,
    #     "profile.default_content_setting_values.javascript": 1,  # Keep JavaScript enabled if needed
    #     "profile.default_content_setting_values.plugins": 2,
    #     "profile.default_content_setting_values.popups": 2,
    #     "profile.default_content_setting_values.geolocation": 2,
    #     "profile.default_content_setting_values.notifications": 2,
    #     "profile.default_content_setting_values.auto_select_certificate": 2,
    #     "profile.default_content_setting_values.fullscreen": 2,
    #     "profile.default_content_setting_values.mouselock": 2,
    #     "profile.default_content_setting_values.mixed_script": 2,
    #     "profile.default_content_setting_values.media_stream": 2,
    # }
    # options.add_experimental_option("prefs", prefs)

    # # Set page load strategy to 'eager' to speed up page loads
    # options.page_load_strategy = "eager"

    # Specify the path to the ChromeDriver
    service = Service("/usr/local/bin/chromedriver")

    # Initialize the WebDriver with the specified service and options
    driver = webdriver.Chrome(service=service, options=options)
    return driver


query = "Bây giờ là mấy giờ"
max_links= 10
max_contents = 2
results_web_searching = []
driver = None
text_web_searching = ""  # Initialize the variable here

try:
    # Initialize the Chrome driver once
    driver = chrome()
    # Remove implicit wait to avoid conflicts with explicit waits
    wait = WebDriverWait(driver, 10)  # Explicit wait

    # Step 1: Fetch links from Google search results
    search_url = f"https://www.google.com/search?q={query}"
    time.sleep(3)
    driver.get(search_url)
    logging.info(f"Navigated to search URL: {search_url}")

    # Wait for search results to load using CSS selector
    wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[jsname="UWckNb"]'))
    )
    link_elements = driver.find_elements(By.CSS_SELECTOR, 'a[jsname="UWckNb"]')
    links = []
    
    for link_element in link_elements[:max_links]:
        href = link_element.get_attribute("href")
        if href:
            links.append(href)
            logging.info(f"Link found: {href}")

    # Step 2: Iterate through the fetched links and get page content
    bad_domains = ["youtube.com"]
    for link in links:
        if any(bad_domain in link for bad_domain in bad_domains):
            logging.info(f"Skipping unwanted domain: {link}")
            continue

        try:
            driver.get(link)
            logging.info(f"Navigated to URL: {link}")

            # Wait for the body of the page to load
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            content = driver.find_element(By.TAG_NAME, "body").text
            if content:
                results_web_searching.append(content)
                logging.info(f"Content fetched from {link}")
        except Exception as e:
            logging.warning(f"Error fetching content from {link}: {e}")
            continue

        if len(results_web_searching) >= max_contents:
            logging.info(f"Reached maximum content limit: {max_contents}")
            break

    # Step 3: Combine the fetched contents
    text_web_searching = "\n\n".join(
        [f'- "{content}"' for content in results_web_searching]
    )
    text_web_searching = text_web_searching[:20000]  # Limit to 20,000 characters
    logging.info("Web searching completed successfully.")

except Exception as general_e:
    logging.error(f"An unexpected error occurred: {general_e}")
    # Optionally, set text_web_searching to an error message or keep it as initialized

# finally:
#     if driver:
#         driver.quit()  # Ensure the driver is properly closed
#         logging.info("Chrome driver has been closed.")

/home/admin-hieunn/important/VNUIS-Chatbot/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2025-01-12 15:19:51,398 - INFO - Navigated to search URL: https://www.google.com/search?q=Bây giờ là mấy giờ
2025-01-12 15:20:01,674 - ERROR - An unexpected error occurred: Message: 

